<a href="https://colab.research.google.com/github/abnas7511/RAG_Apollo11/blob/main/RAG_apollo11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Retrieval Augmented Generation (RAG)**

You're an astronomy student who's very curious about the Apollo 11 missions,
and through your research, you've found a lot of different types of data (otherwise known as multimodal) from NASA's
public archive.

1. Text: You have the full final NASA report post-mission, spanning over 300
pages of incredibly informative content that details a summary of everything
that happened as well as conclusions that NASA researchers and engineers
came to. For the sake of this exercise, we've selected 3 particularly interesting pages, and converted them to images (you'll see soon why).

2. Video: You also have several clips of the famous Neil Armstrong and Buzz Aldrin footage as they
first stepped onto the moon, containing highlights of their moonwalks as well
as raising the American flag.

3. Audio: Finally, you have highlights from the audio recorded throughout the
mission, which provides insights into how communication between the astronauts
occurred as well as from the astronauts to mission control.

Now, you want to search through and summarize this information for your
upcoming research paper. Using your newfound skills from this course, you
can accomplish this using Gemini! In particular, we will build a Retrieval Augmented Generation (RAG) system that you can directly interact with.

In [1]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

In [2]:
import time
from tqdm import tqdm
import pathlib
import google.generativeai as genai
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import pandas as pd
from PIL import Image
import pytesseract
from IPython.display import Markdown

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
!wget -O resources.zip "https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip"

--2024-07-11 18:01:17--  https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip
Resolving video.udacity-data.com (video.udacity-data.com)... 104.19.140.72, 104.19.139.72, 104.19.138.72, ...
Connecting to video.udacity-data.com (video.udacity-data.com)|104.19.140.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286142532 (273M) [application/zip]
Saving to: ‘resources.zip’

resources.zip       100%[===================>] 272.89M   130MB/s    in 2.1s    

2024-07-11 18:01:20 (130 MB/s) - ‘resources.zip’ saved [286142532/286142532]



In [5]:
!unzip resources.zip


Archive:  resources.zip
   creating: resources/
  inflating: __MACOSX/._resources    
   creating: resources/video/
  inflating: __MACOSX/resources/._video  
  inflating: resources/.DS_Store     
  inflating: __MACOSX/resources/._.DS_Store  
   creating: resources/audio/
  inflating: __MACOSX/resources/._audio  
   creating: resources/text/
  inflating: __MACOSX/resources/._text  
  inflating: resources/video/Apollo11PlaqueComparison.mov  
  inflating: __MACOSX/resources/video/._Apollo11PlaqueComparison.mov  
  inflating: resources/video/Apollo11Intro.mov  
  inflating: __MACOSX/resources/video/._Apollo11Intro.mov  
  inflating: resources/video/Apollo11MoonwalkMontage.mov  
  inflating: __MACOSX/resources/video/._Apollo11MoonwalkMontage.mov  
  inflating: resources/video/OneSmallStepCompilation.mov  
  inflating: __MACOSX/resources/video/._OneSmallStepCompilation.mov  
  inflating: resources/video/RaisingTheAmericanFlag.mov  
  inflating: __MACOSX/resources/video/._RaisingTheAmericanFl

In [6]:
data_dir = pathlib.Path("resources/")
all_file_names = [str(file) for file in data_dir.rglob("*") if file.is_file() and not file.name.startswith('.')]

In [7]:
for file_name in all_file_names:
    print(file_name)

print(len(all_file_names))

resources/text/images-023.jpg
resources/text/images-020.jpg
resources/text/images-333.jpg
resources/video/Apollo11PlaqueComparison.mov
resources/video/Apollo11Intro.mov
resources/video/Apollo11MoonwalkMontage.mov
resources/video/BuzzDescendsCompilation.mov
resources/video/RaisingTheAmericanFlag.mov
resources/video/OneSmallStepCompilation.mov
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
14


## Retrieval Augmented Generation (RAG)

To showcase how we build a RAG, we will first build one for the Text case, and generalize it further after. Here is the general idea:
1. **Data Preparation** (done above): We first collected various types of data from NASA's public archive related to the Apollo 11 mission, including text, video, and audio files.
2. **Data Extraction and Summarization**: Extract the multimodal data from images, e.g. extract text from images using Optical Character Recognition (OCR), and use Gemini to generate summaries using a specialized prompt.
3. **Embedding Generation**: Convert the generated summaries into vector embeddings using Gemini's Text Embedding Model. These embeddings represent the summaries in a numerical format suitable for efficient similarity searches.
4. **Creating a Vector Database**: A Vector database was created to store the embeddings. This database facilitates fast and efficient retrieval of relevant documents based on similarity searches. We chose to use Chroma DB.
5. **Querying the RAG System**: For a given query, the system retrieves the most relevant documents (based on their embeddings) and generates a response using the retrieved documents as context.

Something important to note is that RAGs are usually used only when there is a surplus of data. In other words, if the data can't fit into the model prompt. In this case, the data we provided likely can fit into Gemini's 1 million token window, but for the sake of simplicity and restrictions of Google Colab's runtime, we opted to use a smaller set of data.

In [8]:
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

In [9]:
def create_text_summary():
  path = pathlib.Path("resources/text")

  text_summary_prompt = f"""You are an assistant tailored for summarizing text for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw text.
  Give a concise summary of the text that is well optimized for retrieval. Here is the text."""

  images = []
  text_summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue

    image = Image.open(f)
    response = model.generate_content([text_summary_prompt, pytesseract.image_to_string(image)])

    images.append(image)
    text_summaries.append(response.text)

  return images, text_summaries

In [10]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

model = genai.GenerativeModel('models/gemini-1.5-flash', safety_settings=safety_settings)

In [11]:
image_files, text_summaries = create_text_summary()

In [12]:
for text_summary in text_summaries:
  print(text_summary)

This document describes the Apollo 11 mission launch and lunar journey.  It outlines the key events, including lift-off at 9:32 EDT on July 16, 1969, earth orbit insertion, translunar coast, and transpositional docking. The document also details the midcourse corrections and navigation sightings that occurred during the lunar journey. 

This document outlines the Apollo 11 Flight Plan, developed by the Flight Planning Branch with support from TRW Systems. It details the planned operations and crew activities for the AS-506/CSM-107/LM-5 mission, aimed at fulfilling the test objectives defined in the Mission Requirements for a G Type Mission Lunar Landing. The trajectory parameters used are based on a July 16, 1969 launch with a 72° azimuth. Any proposed changes to the Flight Plan must be submitted to the Crew Procedures Control Board (CPCB) for review. The Chief, Flight Planning Branch, will determine the scope of the proposed changes, and Mr. T. A. Guillory will act as the coordinator 

We create the Chroma database using the generated summaries. You might be wondering what Vector DB and Chroma DB are.

**Vector Database**: A specialized database designed to store and manage high-dimensional vectors, which are numerical representations of data points. It allows efficient similarity searches to find vectors (and their corresponding data) that are close to a given query vector.

**Chroma DB**: An implementation of a vector database used to store and retrieve vector embeddings. These embeddings are generated from our summaries and allow us to perform efficient similarity searches.

In [13]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/text-embedding-004'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]

In [14]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [15]:
text_db = create_chroma_db(text_summaries, "text_nasa")

In [16]:
data = {
    'embeddings': text_db.peek()['embeddings'],
    'documents': text_db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.08986429125070572, -0.00667669577524066, 0....",This document describes the Apollo 11 mission ...
1,"[0.0845600813627243, 0.008062985725700855, 0.0...",This document outlines the Apollo 11 Flight Pl...
2,"[0.07686157524585724, 0.00110200559720397, -0....",This document outlines assumptions and ground ...


Now let's actually try querying our information. We'll test a simple

*   List item
*   List item

example like getting some file that has to do with the Apollo 11 Flight Plan.

In [17]:
def get_relevant_files(query, db):
  results = db.query(query_texts=[query], n_results=3)
  return results["ids"][0]

In [18]:
files = get_relevant_files("Apollo 11 Flight Plan", text_db)
print(files)

['1', '0', '2']


### Video and Audio

Congrats! You've successfully built a working RAG for text. Now, let's extend this concept to Video and Audio, and build out some more complex queries. We'll begin by generalizing the above summary creation function to all sorts of modalities.

In [19]:
def create_summary(modality):
  path = data_dir / modality

  summary_prompt = f"""You are an assistant tailored for summarizing {modality} for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw {modality}.
  Give a concise summary of the {modality} that is well optimized for retrieval. Here is the {modality}."""

  files = []
  summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue
    print(f)

    if modality == "text":
      file = Image.open(f)
      response = model.generate_content([summary_prompt, pytesseract.image_to_string(file)])

    else:
      file = genai.upload_file(f)

      while file.state.name == "PROCESSING":
        print("Waiting for video file upload...\n", end='')
        time.sleep(5)
        file = genai.get_file(file.name)

      response = model.generate_content([summary_prompt, file])


    files.append(file)
    summaries.append(response.text)

  return files, summaries

In [21]:
all_files = []
all_summaries = []

for modality_type in ["audio", "text", "video"]:
  files, summaries = create_summary(modality_type)
  all_files.extend(files)
  all_summaries.extend(summaries)

resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/text/images-023.jpg
resources/text/images-020.jpg
resources/text/images-333.jpg
resources/video/Apollo11PlaqueComparison.mov
Waiting for video file upload...
resources/video/Apollo11Intro.mov
Waiting for video file upload...
resources/video/Apollo11MoonwalkMontage.mov
Waiting for video file upload...
resources/video/BuzzDescendsCompilation.mov
Waiting for video file upload...
resources/video/RaisingTheAmericanFlag.mov
Waiting for video file upload...
resources/video/OneSmallStepCompilation.mov
Waiting for video file upload...


In [22]:
db = create_chroma_db(all_summaries, "nasa")

In [23]:
data = {
    'embeddings': db.peek()['embeddings'],
    'documents': db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.009563048370182514, -0.018056461587548256, ...",Two people discuss aircraft instrumentation wh...
1,"[0.04265424609184265, -0.005710171069949865, -...",The audio is a conversation between two people...
2,"[0.04477608948945999, 0.0014422578969970345, -...",Astronauts on the moon describe the surface an...
3,"[0.05467071011662483, -0.03329379856586456, -0...",Buzz Aldrin descends the Apollo 11 lunar modul...
4,"[0.03196701034903526, -0.01753888465464115, -0...",A side by side comparison of the original grai...
5,"[0.028621070086956024, -0.027986012399196625, ...",Neil Armstrong takes his first step on the moo...
6,"[0.024294933304190636, -0.006738951429724693, ...",This audio is a conversation between two peopl...
7,"[0.03903806954622269, -0.015813903883099556, -...","Astronauts on the moon discuss the craters, mo..."
8,"[0.0029723166953772306, 0.021685808897018433, ...",A conversation between two people about an air...
9,"[0.07570424675941467, -0.01430736668407917, -0...",This document details the Apollo 11 mission la...


In [24]:
files = get_relevant_files("communication with Mission Control", db)
print(files)

['6', '1', '0']


Can we do more than just return the most relevant file? Yes we can! We can ask Gemini to return a response to the query using the files it thinks are most relevant, provide an answer and tell us what files it used! This is really exciting, and has vast applications in many industries.

In [25]:
def query_rag(query, db):
    files = get_relevant_files(query, db)
    prompt = [all_files[int(f)] for f in files]
    prompt.append("Generate a response to the query using the provided files. Here is the query.")
    prompt.append(query)
    return model.generate_content(prompt).text, [all_file_names[int(f)] for f in files]

In [26]:
for response in query_rag("Explain what happened with the Apollo 11 Mission.", db):
    print(response)

On July 20th, 1969, the Apollo 11 mission became the first human-crewed spaceflight to land on the Moon.  The mission was captained by Neil Armstrong with Buzz Aldrin as the lunar module pilot and Michael Collins as the command module pilot. Armstrong became the first human to walk on the Moon with Aldrin following him.  The mission launched on July 16th, 1969 and lasted for 8 days, 3 hours, 18 minutes. The crew safely returned to Earth on July 24th, 1969. 
['resources/audio/Apollo11OnboardAudioHighlightClip1.mp3', 'resources/video/Apollo11MoonwalkMontage.mov', 'resources/video/OneSmallStepCompilation.mov']


In [27]:
for response in query_rag("What happens at the Translunar Coast in the Mission Description?", db):
    print(response)

After TLI, the spacecraft is placed in a free lunar return trajectory.  Several major events happen before LOI: transposition, docking, and LM ejection, including SIVB photography. 
['resources/video/Apollo11MoonwalkMontage.mov', 'resources/video/Apollo11PlaqueComparison.mov', 'resources/audio/Apollo11OnboardAudioHighlightClip2.mp3']
